In [1]:
from extract_data import extract_data
import pandas as pd
import numpy as np
from scipy import optimize
from scipy.optimize import linear_sum_assignment
from genSankey import genSankey
import chart_studio.plotly as py
import plotly

import plotly.graph_objs as go

from random import seed
from random import randint

In [2]:
#importing cleaned data
data = extract_data("data","data",1)

In [3]:
#removing don't know's and no votes from the labels, label choice is who people voted in the general election
data['generalElectionVote'] = data['generalElectionVote'].fillna("Don't know") 

list_to_remove = []
for i in range(data.shape[0]):
    if data.loc[i,'generalElectionVote'] == "Don't know":
        list_to_remove.append(i)
    elif data.loc[i,'generalElectionVote'] == "No - none":
        list_to_remove.append(i)
    elif data.loc[i,'generalElectionVote'] == "Plaid Cymru":
        list_to_remove.append(i)
    elif data.loc[i,'generalElectionVote'] == "Scottish National Party (SNP)":
        list_to_remove.append(i)
    
data = data.drop(list_to_remove)
data = data.reset_index(drop=True)



In [4]:
#importing k-modes required packages
from kmodes.kmodes import KModes, labels_cost
from sklearn.utils import check_random_state
from sklearn.utils.validation import check_array
from collections import defaultdict
from kmodes.util.dissim import matching_dissim, ng_dissim
from kmodes.util.__init__ import  pandas_to_numpy, encode_features

In [5]:
#function to find the cost matrix of the cluster assignments
def cost_matrices(X, centroids, dissim, membship=None):
   
    X = check_array(X)
    n_centroids = centroids.shape[0]
    n_points = X.shape[0]
    cost = np.empty(shape=[n_points,n_centroids], dtype=np.uint16)
    labels = np.empty(n_points, dtype=np.uint16)
    for ipoint, curpoint in enumerate(X):
        diss = dissim(centroids, curpoint, X=X, membship=membship)
        cost[ipoint,:]=diss
        clust = np.argmin(diss)
        labels[ipoint] = clust
    

    return cost

In [6]:
#converting data to matrices and dropping certain features including the labels
k_modes_data = data.drop(['generalElectionVote','enviroGrowth','immigCultural','immigSelf','immigEcon','EUIntegrationSelf','happyEULeave'],axis=1)
k_modes_data_matrix = k_modes_data.to_numpy()


In [73]:
n_clusters = 6
kmodes = KModes(n_jobs = -1, n_clusters = 6, init = 'Huang', random_state = 0)
kmodes.fit_predict(k_modes_data_matrix)

array([2, 3, 0, ..., 1, 3, 9], dtype=uint16)

In [108]:

def purity_check(labels,data):
    purity_check = pd.DataFrame({'Cluster Labels' : labels})
    purity_check['Vote'] = data['generalElectionVote']
    cluster_purity_list=[]
    for cluster in range(n_clusters):
    
        vote_list = []
        for x in range(purity_check.shape[0]):
            if purity_check.loc[x,'Cluster Labels'] == cluster:
                vote_list.append(data.loc[x,'generalElectionVote'])
        vote_list_data_frame = pd.DataFrame({'Assignment':vote_list})
        cluster_purity_list.append(max(vote_list_data_frame['Assignment'].value_counts(normalize=True)))

    mean_purity = np.mean(cluster_purity_list)
    return mean_purity
    
    
            
    
                
            

0.5295544171178856


In [74]:
#calculating the cost matrix
X = pandas_to_numpy(k_modes_data_matrix)
X = check_array(X, dtype=None)
X, _ = encode_features(X, enc_map=kmodes._enc_map)
costs = cost_matrices(X,kmodes._enc_cluster_centroids,matching_dissim)

In [75]:
#computing the linear sum assignments
def optimal_cluster_assignments_lists(optimal_cluster_assignments,stage=None):
    optimal_cluster_assignments_names = []
    optimal_cluster_assignments_number = []
    for i in range(optimal_cluster_assignments[0].shape[0]):
        name = data.loc[optimal_cluster_assignments[0][i],'generalElectionVote']
        cluster_number = optimal_cluster_assignments[1][i]
        name_count = 0 
        for x in range(len(optimal_cluster_assignments_names)):
                if name in optimal_cluster_assignments_names[x]:
                    name_count += 1
        if name_count != 0:
            name = name + str(name_count+1)
        if stage != None:
            print(name)
            name = name + "_stage_" + str(stage)
        optimal_cluster_assignments_names.append(name)
        optimal_cluster_assignments_number.append(cluster_number)
    return optimal_cluster_assignments_names, optimal_cluster_assignments_number


first_optimal_assignments = linear_sum_assignment(costs)
first_cluster_assignments_names, first_cluster_assignments_number = optimal_cluster_assignments_lists(first_optimal_assignments)
    
    

In [76]:
print(first_cluster_assignments_number)
print(first_cluster_assignments_names)

[11, 12, 3, 8, 2, 9, 10, 0, 1, 4, 6, 5, 7]
['Labour', 'I would/did not vote', 'Liberal Democrat', 'Other', 'Conservative', 'Conservative2', 'Conservative3', 'Conservative4', 'Conservative5', 'Labour2', 'Liberal Democrat2', 'Green Party', 'Labour3']


In [105]:
#creating a data frame of cluster assignments and theyre original parties
cluster_assignments = pd.DataFrame({"Party Vote" : data.loc[:,'generalElectionVote'] })
cluster_assignments['First Assignment'] = kmodes.labels_



0     3637
9     3114
1     3057
5     2987
7     2698
2     2524
3     2356
8     1795
4     1568
11    1397
12    1281
10    1179
6      774
Name: First Assignment, dtype: int64

In [14]:
#replacing the numeric cluster assignment with the optimal name
for i in range(len(first_cluster_assignments_number)):
    cluster_assignments['First Assignment'] = cluster_assignments['First Assignment'].replace(int(first_cluster_assignments_number[i]),first_cluster_assignments_names[i])



In [15]:
#performing reassignments - removing a cluster, at the moment this is done manually
kmodes._enc_cluster_centroids = np.delete(kmodes._enc_cluster_centroids,[0,1,2,6],0) # deleting the second cluster

second_assignment = kmodes.predict(k_modes_data_matrix)



In [16]:
cluster_assignments['Second Assignment'] = second_assignment

second_costs = cost_matrices(X,kmodes._enc_cluster_centroids,matching_dissim)

second_optimal_assignments = linear_sum_assignment(second_costs)

second_cluster_assignments_names, second_cluster_assignments_number = optimal_cluster_assignments_lists(second_optimal_assignments,2)


for i in range(len(second_cluster_assignments_number)):
    cluster_assignments['Second Assignment'] = cluster_assignments['Second Assignment'].replace(int(second_cluster_assignments_number[i]),second_cluster_assignments_names[i])


Liberal Democrat
Conservative
Conservative2


In [17]:
cluster_assignments['Second Assignment'].unique()

array(['Conservative_stage_2', 'Liberal Democrat_stage_2',
       'Conservative2_stage_2'], dtype=object)

In [18]:
index_count = []

for i in range(cluster_assignments.shape[0]):
    index_count.append(i)

cluster_assignments['index'] = index_count
    

In [19]:
figure = genSankey(cluster_assignments,cat_cols=['First Assignment','Second Assignment'],value_cols='index',title='Movement of voters after drop out')

plotly.offline.plot(figure, validate=False)

'temp-plot.html'

In [ ]:
sum(cluster_assignments['First Assignment']=='Conservative')

7327

In [78]:
from sklearn.cluster import KMeans

n_clusters = 6

kmeans = KMeans(n_clusters, random_state=0).fit(X)

predictions = kmeans.predict(X)

labels = data['generalElectionVote']

kmodes._enc_map





[{'Brexit Party': 0,
  'Conservative': 1,
  "Don't know": 2,
  'Green Party': 3,
  'Labour': 4,
  'Liberal Democrat': 5,
  'No - none': 6,
  'Other': 7,
  'Plaid Cymru': 8,
  'Scottish National Party (SNP)': 9,
  'United Kingdom Independence Party (UKIP)': 10},
 {"Don't know": 0,
  'It is completely necessary': 1,
  'It is completely unnecessary': 2,
  'It is important but not absolutely  necessary': 3,
  'It is not necessary but it would be desirable': 4},
 {'Agree': 0,
  'Disagree': 1,
  "Don't know": 2,
  'Neither agree nor disagree': 3,
  'Strongly agree': 4,
  'Strongly disagree': 5},
 {"Don't know": 0,
  'Getting a little better': 1,
  'Getting a little worse': 2,
  'Getting a lot better': 3,
  'Getting a lot worse': 4,
  'Staying about the same': 5},
 {"Don't know": 0,
  'Got a little better': 1,
  'Got a little worse': 2,
  'Got a lot better': 3,
  'Got a lot worse': 4,
  'Stayed the same': 5},
 {"Don't know": 0,
  'Got a little better': 1,
  'Got a little worse': 2,
  'Got a l

In [68]:
def kmeans_distance(cluster,data_compare,n_clusters):
    cost_matrix = np.zeros([data_compare.shape[0],n_clusters])
    for i in range(data_compare.shape[0]):
        point_costs = []
        for c in range(n_clusters):
            diss = np.linalg.norm(data_compare[i,:]-cluster[c,:])
            cost_matrix[i,c]=diss
    return cost_matrix

            
            
    

In [71]:
kmeans_cost = kmeans_distance(kmeans.cluster_centers_,X,n_clusters)


In [72]:
k_means_assignments = linear_sum_assignment(kmeans_cost)
k_means_assignments_names, k_means_assignments_number = optimal_cluster_assignments_lists(k_means_assignments)

In [102]:
kmeans.cluster_centers_ = np.delete(kmeans.cluster_centers_,0,0)


In [103]:
kmeans.cluster_centers_.shape



(5, 32)